Generate Functional Connectivity Gradients

This requires Brainspace - https://brainspace.readthedocs.io/en/latest/

In [ ]:
import numpy as np
import nibabel as nib
from sklearn.decomposition import PCA
from brainspace.gradient import GradientMaps
import os
import pickle

Add the following file paths

In [ ]:
sub_mask_path = '/HarvardOxford_Subcortical_Mask.nii.gz' # path to subcortical mask
gm_mask_path = '/Gray_Matter_Mask.nii.gz' # path to grey matter mask
data_path = '/3T_fMRI_data/' # path to data
save_path = '/3T-gradients/' # path to save output

example_pt_path = os.path.join(data_path, os.listdir(data_path)[0]) # should be .nii.gz or .nii file

Load masks and patient fMRI

In [ ]:
sub_mask = nib.load(sub_mask_path).get_fdata()
gm_mask = nib.load(gm_mask_path).get_fdata()

fmri_data = nib.load(example_pt_path).get_fdata()

Compute Gradient - following https://doi.org/10.1016/j.neuroimage.2017.06.075

Extract fMRI time series data in subcortical voxels

In [ ]:
A = fmri_data[sub_mask==1,:].transpose()

Extract fMRI time series data in grey matter voxels and reduce dimensionality using PCA

In [ ]:
B = fmri_data[gm_mask==1,:].transpose()
pca = PCA(n_components=B.shape[0]-1)
B_filt = pca.fit_transform(B)

Correlate A (subcortical voxels) and B (grey matter voxels) to get C; Filter C to maintain only top 10% connections in each row

In [ ]:
def filter_C(C):
    C_filt = np.zeros(C.shape)
    for i in range(C.shape[0]):
        a = C[i,:]
        thresh = np.sort(a)[::-1][int(0.11*len(a))]
        C_filt[i,a>thresh] = a[a>thresh]
    return C_filt

In [ ]:
C = np.arctanh(np.corrcoef(A, B_filt, rowvar=False)[:A.shape[1], A.shape[1]:])
C_filt = filter_C(C)

Calculate Similarity Matrix and Gradient Maps using Brainspace

In [ ]:
similarity_method = 'pearson' # {'pearson', 'spearman', 'cosine', 'normalized_angle', 'gaussian'}
gradient_method = 'dm' # {'dm', 'le', 'pca'} for [diffusion maps, laplacian eigenmaps, pca]

gm = GradientMaps(n_components=250, approach=gradient_method, kernel=similarity_method, alignment=None)
gm.fit(C_filt)

out_dict = {'gradient': gm.gradients_, 'lambda': gm.lambdas_}

with open(os.path.join(save_path, gradient_method + '_' + similarity_method + '_grad.pickle'), 'wb') as f:
    pickle.dump(out_dict, f)